## 1. Introduction

The dataset used in this analysis is provided by **Argyris Anastasolpoulos**.  
It contains listings of residential properties in Greece for the year 2022, covering the regions of **Attiki** and **Thessaloniki**.

**Dataset**: [Greece Property Listings](https://www.kaggle.com/datasets/argyrisanastopoulos/greece-property-listings)  
**Source**: Kaggle  


## 2. Data Loading
Load the dataset and preview the structure of the data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import (
    mean_absolute_error,
    explained_variance_score,
    median_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

df = pd.read_csv("/kaggle/input/greece-property-listings/greece_listings.csv")

**Original Data Shape**

In [ ]:
df
pd.set_option('display.max_columns', None)

In [ ]:
df.shape
df.info()
df.head()
df.describe()


## 3. Data Preprocessing

**We remove the houses located outside Attica, specifically in Thessaloniki, since there are only 1,187 compared to 18,813 in Attica.**

In [ ]:
attica_count = df[df["location_region"] == "Αττική"].shape[0]
print(f"Number of properties in Αττική: {attica_count}")
df = df[df["location_region"] == "Αττική"].reset_index(drop=True)

**We print the distinct regions of Attica; later, we will create separate fields for each region.**

In [ ]:
unique_locations = df["location_name"].unique()
print(f"Number of unique locations: {len(unique_locations)}")
df["location_name"].value_counts()

**We keep the regions with more than 100 houses, so the model can learn consistent patterns without overfitting to areas with limited data.**

In [ ]:
loc_df = df["location_name"].value_counts().reset_index()
loc_df.columns = ["location_name", "count"]
locations_to_keep = loc_df[loc_df["count"] >= 100]["location_name"]
df = df[df["location_name"].isin(locations_to_keep)].reset_index(drop=True)

unique_locations = df["location_name"].unique()
print(f"Number of unique locations with more than 100 houses: {len(unique_locations)}")

**We convert the region names into Latin characters in order to create new separate fields for each region. This process is called one-hot encoding and transforms an alphanumeric feature into multiple boolean features. These new features are much easier for the model to handle.**

In [ ]:
greek_to_english_map = {
    "Αθήνα": "athina",
    "Αχαρνές": "acharnes",
    "Καλλιθέα": "kallithea",
    "Βάρη - Βούλα - Βουλιαγμένη": "vari-voula-vouliagmeni",
    "Πειραιάς": "peiraias",
    "Χαλάνδρι": "chalandri",
    "Γλυφάδα": "glyfada",
    "Άλιμος": "alimos",
    "Μαρούσι": "marousi",
    "Νίκαια": "nikaia",
    "Διόνυσος": "dionysos",
    "Ίλιον": "ilion",
    "Παλαιό Φάληρο": "palaio-faliro",
    "Νέα Σμύρνη": "nea-smyrni",
    "Ζωγράφου": "zografou",
    "Ηλιούπολη": "ilioupoli",
    "Βύρωνας": "vyronas",
    "Νέα Ιωνία": "nea-ionia",
    "Γαλάτσι": "galatsi",
    "Περιστέρι": "peristeri",
    "Αργυρούπολη": "argyroupoli",
    "Λαυρεωτική": "lavreotiki",
    "Κρωπία": "kropia",
    "Ελληνικό - Αργυρούπολη": "elliniko-argyroupoli",
    "Μοσχάτο - Ταύρος": "moschato-tavros",
    "Παγκράτι": "pagkrati",
    "Άγιος Δημήτριος": "agios-dimitrios",
    "Αιγάλεω": "aigaleo",
    "Άγιοι Ανάργυροι": "agioi-anargyroi",
    "Δάφνη": "dafni",
    "Ραφήνα Πικέρμι": "rafina-pikermi",
    "Σπάτα": "spata",
    "Κερατσίνι-Δραπετσώνα": "keratsini-drapetsona",
    "Χολαργός": "cholargos",
    "Φιλοθέη - Ψυχικό": "filothei-psychiko"
}
df["location_name_eng"] = df["location_name"].map(greek_to_english_map)
df["location_name_eng"] = df["location_name_eng"].fillna("unknown")

location_name_eng = df["location_name_eng"].unique()

for loc in location_name_eng:
    df[loc] = 0

for loc in location_name_eng:
    df.loc[df["location_name_eng"] == loc, loc] = 1

region_counts = {}

for loc in location_name_eng:
    count_ones = df[loc].sum()
    region_counts[loc] = count_ones
    print(f"Region '{loc}': {count_ones} houses")

total_ones = sum(region_counts.values())
total_rows = df.shape[0]

print(f"\nTotal houses counted in one-hot columns: {total_ones}")
print(f"Total rows in dataset: {total_rows}")

assert total_ones == total_rows, "Mismatch between one-hot counts and total rows!"



**The conversion was successful, without errors (for example, a house mistakenly belonging to 2 areas).
The next step is to delete the fields related to the name of the area that are no longer useful, so that the dataset is clean and easily readable.**

**Let’s check for res_address if there are specific areas or neighborhoods with more than 100 houses that could be made into separate fields and are not located in the center of Athens.**

In [ ]:
res_address_counts = df["res_address"].value_counts()
res_address_over_100 = res_address_counts[res_address_counts > 100]

unique_locations = df["location_name"].unique()

missing_parts_counts = {}

for address in res_address_over_100.index:
    parts = [p.strip() for p in address.split(",")]
    
    mask = (df["res_address"] == address) & (df["location_name"] != "Αθήνα")
    count_filtered = mask.sum()
    
    for part in parts:
        if part not in unique_locations and count_filtered > 0:
            missing_parts_counts[part] = missing_parts_counts.get(part, 0) + count_filtered

print("Regions not in location_name that dont have location_name='Αθήνα':")
for part, count in missing_parts_counts.items():
    print(f"{part}: {count}")


**Separate areas: Ekali, Vrilissia, Vari, Varkiza, Drosia, Kaminia (relatively larger or independent administrative areas).
We create new fields for these areas and make sure to remove them from the broader area they have belonged to so far. For example, houses in Ekali and Drosia belong to the area of Dionysos. To avoid a house belonging to two areas, we will remove the '1' from the Dionysos area.**

**Also, there is the case where the name of one area appears together with the name of another in the res_address field.
As shown below, the alphanumeric string "Vari, Varkiza" appears, which can cause confusion because we will create a separate field for Vari and a separate one for Varkiza. In such a case, we will change the string to "Varkiza."
The same happens with "Piraeus, Kaminia."**

In [ ]:
df["res_address"] = df["res_address"].str.replace("Βάρη,Βάρκιζα", "Βάρκιζα", regex=False)
df["res_address"] = df["res_address"].str.replace("Πειραιάς,Καμίνια", "Καμίνια", regex=False)

In [ ]:
greek_areas = ["Εκάλη", "Βριλήσσια", "Βάρη", "Βάρκιζα", "Δροσιά", "Καμίνια"]

def extract_main_area(address):
    for area in greek_areas:
        if area in address:
            return area
    return address

df["res_address"] = df["res_address"].apply(lambda x: extract_main_area(x) if pd.notnull(x) else x)

mask_after = df["res_address"].dropna().apply(lambda x: any(area in x for area in greek_areas))

unique_cleaned = df.loc[mask_after, "res_address"].unique()

for addr in sorted(unique_cleaned):
    print(addr)


**We create a temporary field '6new_temp' that stores the area extracted from the res_address field.**

In [ ]:
area_map = {
    "Εκάλη": "ekali",
    "Βριλήσσια": "vrilissia",
    "Βάρη": "vari",
    "Βάρκιζα": "varkiza",
    "Δροσιά": "drosia",
    "Καμίνια": "kaminia"
}

def assign_area(address):
    if pd.isna(address):
        return ''
    for greek_name, eng_name in area_map.items():
        if greek_name in address:
            return eng_name
    return ''

df['6new_temp'] = df['res_address'].apply(assign_area)

**We make sure to remove the 1 from the general area to which the house belonged.**

In [ ]:
df.loc[df["6new_temp"] == "vari", "vari-voula-vouliagmeni"] = 0
df.loc[df["6new_temp"] == "vari", "kropia"] = 0
df.loc[df["6new_temp"] == "vari", "vari"] = 1

df.loc[df["6new_temp"] == "varkiza", "vari-voula-vouliagmeni"] = 0
df.loc[df["6new_temp"] == "varkiza", "kropia"] = 0
df.loc[df["6new_temp"] == "varkiza", "varkiza"] = 1

df.loc[df["6new_temp"] == "ekali", "dionysos"] = 0
df.loc[df["6new_temp"] == "ekali", "acharnes"] = 0
df.loc[df["6new_temp"] == "ekali", "ekali"] = 1

df.loc[df["6new_temp"] == "vrilissia", "chalandri"] = 0
df.loc[df["6new_temp"] == "vrilissia", "marousi"] = 0
df.loc[df["6new_temp"] == "vrilissia", "vrilissia"] = 1

df.loc[df["6new_temp"] == "drosia", "dionysos"] = 0
df.loc[df["6new_temp"] == "drosia", "drosia"] = 1

df.loc[df["6new_temp"] == "kaminia", "peiraias"] = 0
df.loc[df["6new_temp"] == "kaminia", "kaminia"] = 1


In [ ]:
target_columns = ['ekali', 'vrilissia', 'vari', 'varkiza', 'drosia', 'kaminia']

df[target_columns] = df[target_columns].fillna(0).astype(int)

**We check if there is any ‘double’ area anywhere.**

In [ ]:
regions = ['palaio-faliro', 'acharnes', 'pagkrati', 'athina', 'ilion', 'galatsi', 'dafni', 'kallithea',
           'argyroupoli', 'peristeri', 'kropia', 'nea-ionia', 'moschato-tavros', 'vari-voula-vouliagmeni',
           'nikaia', 'peiraias', 'zografou', 'chalandri', 'ilioupoli', 'dionysos', 'alimos', 'marousi',
           'nea-smyrni', 'aigaleo', 'glyfada', 'spata', 'keratsini-drapetsona', 'agioi-anargyroi',
           'agios-dimitrios', 'rafina-pikermi', 'lavreotiki', 'elliniko-argyroupoli', 'vyronas', 'cholargos',
           'filothei-psychiko', 'ekali', 'vrilissia', 'vari', 'varkiza', 'drosia', 'kaminia']

In [ ]:
total_ones = df[regions].sum().sum()
print(f"Total 1 in all areas: {total_ones}")
print(f"Rows in the dataset: {len(df)}")

**The fields that describe the area with alphanumeric strings are no longer needed, since we successfully converted the areas using one-hot encoding.**

In [ ]:
df.drop(columns=["location_name"], inplace=True)
df.drop(columns=["location_name_eng"], inplace=True)
df.drop(columns=["location_region"], inplace=True)
df.drop(columns=["res_address"], inplace=True)
df.drop(columns=["6new_temp"], inplace=True)


**The creation date of the listing and whether it has been deleted or not and when is also not useful**

In [ ]:
df.drop(columns=["res_date"], inplace=True)
df.drop(columns=["deleted"], inplace=True)
df.drop(columns=["deleted_at"], inplace=True)

**Below we see the distribution of the "res_type" field which shows the type of property being sold**

In [ ]:
res_type_counts = df['res_type'].value_counts().reset_index()
res_type_counts.columns = ['res_type', 'count']

plt.figure(figsize=(10, 6))
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=res_type_counts,
    x='res_type',
    y='count',
    palette='viridis'
)


plt.title('Distribution of res_type')
plt.xlabel('res_type')
plt.ylabel('Count')
plt.xticks(rotation=45)

for i, row in res_type_counts.iterrows():
    ax.text(i, row['count'] + 0.5, str(row['count']), ha='center', va='bottom')

plt.tight_layout()
plt.show()


**The types "Building" and "House" make up only 4.6% of the "res_type" field. These categories have much lower frequency compared to the others and relatively vague characteristics, making reliable analysis or modeling difficult. For this reason, they were removed from the dataset.**

In [ ]:
df = df[~df['res_type'].isin(['Κτίριο', 'Οικία'])]

In [ ]:
df

**The field construction_year indicates the year the property was built. For a more useful representation, it will be converted to house_age, meaning the age of the property in years (current year - construction year). This variable is likely to be more relevant to the price, as newer or older properties may exhibit different patterns in terms of their value.**

In [ ]:
df['house_age'] = (2022 - df['construction_year']).astype('Int64')
df.drop(columns=["construction_year"], inplace=True)
counts = df['house_age'].value_counts().sort_index()

print("Discrete values of house_age:")
print(counts)

**Checking fields for missing values**

In [ ]:
print(df.isna().sum().head(19))


**The parking field is missing 78% of its values, so it will undoubtedly be removed.**

In [ ]:
df.drop(columns=["parking"], inplace=True)

**In the bathrooms field, before handling the NaN values, we will perform an analysis to observe the variation of the values in the bathrooms field.**

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

summary = df.groupby('bathrooms')['res_price'].agg(['mean', 'count']).sort_index()

print(summary)

**As we can observe, properties with 6 or more bathrooms are found in fewer than 100 homes, so we will remove these entries to avoid overfitting.**

In [ ]:
df['bathrooms'] = df['bathrooms'].fillna(0).astype(int)


In [ ]:
df = df[df['bathrooms'] < 6]

summary = df.groupby('bathrooms')['res_price'].agg(['mean', 'count']).sort_index()
print(summary)


**To fill in the NaN values (which were set to 0 for easier handling, since there were no houses with bathrooms = 0) in the bathrooms column, we will use the property price (res_price) and categorize each property based on the closest average price per number of bathrooms. For example, if a property with a NaN in the bathrooms column is priced at €790,000, we will assign it to the category with 3 bathrooms if that has the closest average price. This method is more reasonable than simply filling in the overall average number of bathrooms, which would lead to many incorrect entries.**

In [ ]:
df = df.copy()

mean_prices = df[df['bathrooms'] != 0].groupby('bathrooms')['res_price'].mean().sort_index()

bins = [-float('inf')]
labels = []

for i in range(len(mean_prices) - 1):
    midpoint = (mean_prices.iloc[i] + mean_prices.iloc[i + 1]) / 2
    bins.append(midpoint)
    labels.append(int(mean_prices.index[i]))

labels.append(int(mean_prices.index[-1]))
bins.append(float('inf'))

df['temp_bathroom'] = pd.cut(df['res_price'], bins=bins, labels=labels, right=False)

df.loc[df['bathrooms'] == 0, 'bathrooms'] = df.loc[df['bathrooms'] == 0, 'temp_bathroom']

df.drop(columns='temp_bathroom', inplace=True)
df['bathrooms'] = df['bathrooms'].astype(int)

print("Houses per number of bathrooms:")
print(df['bathrooms'].value_counts().sort_index())


**We observe that the total number of houses in each category has increased. We follow the exact same logic for the bedrooms as well.**

In [ ]:
df['bedrooms'] = df['bedrooms'].fillna(0).astype(int)


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

summary = df.groupby('bedrooms')['res_price'].agg(['mean', 'count']).sort_index()

print(summary)


**Let’s not forget that the 0 bedrooms value is temporary and represents the NaN values!**

In [ ]:
df = df[df['bedrooms'] < 7]

summary = df.groupby('bedrooms')['res_price'].agg(['mean', 'count']).sort_index()
print(summary)

In [ ]:
df = df.copy()

mean_prices = df[df['bedrooms'] != 0].groupby('bedrooms')['res_price'].mean().sort_index()

bins = [-float('inf')]
labels = []

for i in range(len(mean_prices) - 1):
    midpoint = (mean_prices.iloc[i] + mean_prices.iloc[i + 1]) / 2
    bins.append(midpoint)
    labels.append(int(mean_prices.index[i]))

labels.append(int(mean_prices.index[-1]))
bins.append(float('inf'))

df['temp_bedroom'] = pd.cut(df['res_price'], bins=bins, labels=labels, right=False)

df.loc[df['bedrooms'] == 0, 'bedrooms'] = df.loc[df['bedrooms'] == 0, 'temp_bedroom']

df.drop(columns='temp_bedroom', inplace=True)
df['bedrooms'] = df['bedrooms'].astype(int)

print("Number of houses per number of bedrooms")
print(df['bedrooms'].value_counts().sort_index())


**For the fields that have very few missing values, such as 'res_price_sqr' and 'levels', with 1 and 6 missing entries respectively, we will simply remove those entries.**

In [ ]:
df = df.dropna(subset=['res_price_sqr'])
df = df.dropna(subset=['levels'])


In [ ]:
print(df.isna().sum().head(19))

**Now, only the status and energyclass fields remain. Let's visualize how many values each category has in these two features.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_value_counts(column):
    counts = df[column].value_counts(dropna=False)
    plt.figure(figsize=(10,6))
    ax = sns.barplot(x=counts.index.astype(str), y=counts.values)
    plt.title(f'Distribution for {column}')
    plt.ylabel('Total')
    plt.xlabel(column)
    plt.xticks(rotation=45, ha='right')
    
    for i, v in enumerate(counts.values):
        ax.text(i, v + max(counts.values)*0.01, str(v), ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

plot_value_counts('status')

plot_value_counts('energyclass')


**For the status field, it is observed that there is a value "Άλλη Κατάσταση" ("Other Status") indicating that the property's status is not available. The nan values, which represent missing entries, follow a similar logic. Therefore, "Άλλη Κατάσταση" will be renamed to "Άγνωστη Κατάσταση" ("Unknown Status") and will also include the nan values.**

In [ ]:
df['status'] = df['status'].replace('Άλλη κατάσταση', 'Άγνωστη Κατάσταση')
df['status'] = df['status'].fillna('Άγνωστη Κατάσταση')

**For the energyclass field, along the same lines, the nan values will be merged with the entries labeled "Εκρεμμεί" ("Pending"), which indicate unknown energy status, just like the nan values. This combined category will be renamed to "Άγνωστη Κλάση" ("Unknown Class"). Additionally, the entries with the value "Εξαιρείται" ("Excluded") will be removed since they only account for 44 cases.**

In [ ]:
df['energyclass'] = df['energyclass'].replace('Εκρεμμεί', 'Άγνωστη Κλάση')
df['energyclass'] = df['energyclass'].fillna('Άγνωστη Κλάση')

df = df[df['energyclass'] != 'Εξαιρείται']

**For both status and energyclass, we will apply one-hot encoding so that the information is represented in boolean form instead of as text labels.**

In [ ]:
print(sorted(df['energyclass'].dropna().unique()))
print(sorted(df['status'].dropna().unique()))


In [ ]:
energyclass_map = {
    'Α+': ['a+_class'],
    'Α': ['a_class'],
    'Β+': ['b+_class'],
    'Β': ['b_class'],
    'Γ': ['c_class'],
    'Δ': ['d_class'],
    'Ε': ['e_class'],
    'Ζ': ['z_class'],
    'Η': ['h_class'],
    'Μη αποδοτικό': ['non_efficient_class'],
    'Άγνωστη Κλάση': ['class_unknown_class']
}

for col in set(sum(energyclass_map.values(), [])):
    df[col] = 0
    df[col] = df[col].astype(int)

def assign_energyclass_fields(row):
    for key in energyclass_map.get(row['energyclass'], []):
        row[key] = 1
    return row

df = df.apply(assign_energyclass_fields, axis=1)

In [ ]:
status_map = {
    'Άριστη': ['excellent'],
    'Καλή': ['good'],
    'Ανακαινισμένο': ['renovated'],
    'Νεόδμητο': ['brand_new'],
    'Υπο κατασκευή': ['under_construction'],
    'Χρήζει ανακαίνισης': ['needs_renovation'],
    'Ημιτελές': ['unfinished'],
    'Άγνωστη Κατάσταση': ['unknown_status']
}

for col in set(sum(status_map.values(), [])):
    df[col] = 0
    df[col] = df[col].astype(int)

def assign_status_fields(row):
    for key in status_map.get(row['status'], []):
        row[key] = 1
    return row

df = df.apply(assign_status_fields, axis=1)


**We check if the new fields were created successfully, and then we remove the original ones.**

In [ ]:
df.head(5)

In [ ]:
df.drop(columns=["status"], inplace=True)
df.drop(columns=["energyclass"], inplace=True)

**Checking the levels field, we observe that many entries refer to properties spanning more than one floor. Additionally, there are many different floor combinations. To reduce complexity, we will keep only the values that appear in more than 100 entries, just as we did for the areas field.**

In [ ]:
value_counts = df['levels'].value_counts()

valid_levels = value_counts[value_counts > 100].index

df_filtered = df[df['levels'].isin(valid_levels)].copy()

print(f"Total before: {len(df)}")
print(f"Total after: {len(df_filtered)}")
print("Discrete values left:", list(df_filtered['levels'].unique()))

In [ ]:
df = df_filtered
print("Distribution of discrete values left:")

distribution = df['levels'].value_counts().sort_index()

for level, count in distribution.items():
    print(f"{level}: {count}")


**We will follow the same approach used for the areas, creating new boolean fields to categorize the floors on which the property is located. Each corresponding field will have a value of 1 if the property is on that floor, and 0 otherwise. If a property spans multiple floors, it will have 1s in multiple fields. This way, the model gains more detailed information.**

In [ ]:
floor_map = {
    '1ος': ['1_floor'],
    '2ος': ['2_floor'],
    '3ος': ['3_floor'],
    '3ος,4ος': ['3_4_floor'],
    '4ος': ['4_floor'],
    '4ος,5ος': ['4_5_floor'],
    '5ος': ['5_floor'],
    '5ος,6ος': ['5_6_floor'],
    '6ος': ['6_floor'],
    '7ος': ['7_floor'],
    'Ημιυπόγειο': ['semi_basement'],
    'Ημιώροφος': ['semi_floor'],
    'Ισόγειο': ['0_floor'],
    'Ισόγειο,1ος': ['0_1_floor'],
    'Ισόγειο,1ος,2ος': ['0_1_2_floor'],
    'Υπερυψωμένο': ['elevated']
}


In [ ]:
for floor_level in set(sum(floor_map.values(), [])):
    df[floor_level] = 0 
    df[floor_level] = df[floor_level].astype(int) 

In [ ]:
def assign_floors(row):
    levels = row['levels']
    floors = floor_map.get(levels, [])
    for f in floors:
        row[f] = 1
    return row

df = df.apply(assign_floors, axis=1)

**Now we can remove the levels field that contained the floor information in string format.**

In [ ]:
df.drop(columns=["levels"], inplace=True)

In [ ]:
df

**We observe that approximately 3.83% of the entries lack information about the property's age. Since age is an important factor affecting the price, it would not be appropriate to fill in these missing values with the average or any other simplistic method. Given that the percentage is relatively small, we choose to remove these entries from the dataset.**

In [ ]:
total = len(df)
na_count = df['house_age'].isna().sum()
print(f"NaN values: {na_count} ({na_count / total:.2%} of total)")
df = df[df['house_age'].notna()].copy()

**For the type of residence, represented by the res_type field, we will once again apply one-hot encoding.**

In [ ]:
distribution = df['res_type'].value_counts().sort_index()

for level, count in distribution.items():
    print(f"{level}: {count}")

In [ ]:
type_map = {
    'Διαμέρισμα': ['diamerisma'],
    'Μεζονέτα': ['mezoneta'],
    'Μονοκατοικία': ['monokatoikia']
}

In [ ]:
for types in set(sum(type_map.values(), [])):
    df[types] = 0  # αρχικά 0
    df[types] = df[types].astype(int)

In [ ]:
def assign_types(row):
    types = row['res_type']
    house_type = type_map.get(types, [])
    for f in house_type:
        row[f] = 1
    return row

df = df.apply(assign_types, axis=1)

In [ ]:
df.drop(columns=["res_type"], inplace=True)

**We've cleaned all the alphanumeric fields; now we'll perform a check for possible errors during the transformation—such as having a 1 in both the 'diamerisma' and 'mezoneta' fields simultaneously, which would be incorrect.**

In [ ]:
res_type_columns = ['diamerisma', 'mezoneta', 'monokatoikia']
status_columns = ['excellent', 'renovated', 'unfinished', 'good', 'brand_new', 'under_construction', 'needs_renovation', 'unknown_status']
energyclass_columns = ['a+_class', 'a_class', 'b_class', 'b+_class', 'c_class', 'd_class', 'e_class', 'z_class', 'h_class', 'non_efficient_class', 'class_unknown_class']

def check_single_ones(df, columns, group_name):
    invalid = df[columns].sum(axis=1) != 1
    count_invalid = invalid.sum()
    print(f"{group_name}: {count_invalid} lines with NOT exactly one '1'")

check_single_ones(df, res_type_columns, "res_type")
check_single_ones(df, status_columns, "status")
check_single_ones(df, energyclass_columns, "energyclass")


**The res_price_sqr field is removed because it gives the model direct access to the value it needs to predict, as it’s calculated by res_price_sqr = res_price / res_sqr. If res_price_sqr remains among the input features, the model essentially has the answer in a different form. This causes data leakage and overfitting—resulting in artificially high training performance but poor generalization on new data.**

In [ ]:
y = df['res_price']

X_with_leak = df.drop(columns=['res_price'])

X_no_leak = df.drop(columns=['res_price', 'res_price_sqr'])

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_estimators=50)

r2_with_leak = cross_val_score(model, X_with_leak, y, scoring='r2', cv=5).mean()
r2_no_leak = cross_val_score(model, X_no_leak, y, scoring='r2', cv=5).mean()

print(f"R² no leakage: {r2_no_leak:.5f}")
print(f"R² with leakage (res_price_sqr): {r2_with_leak:.5f}")


In [ ]:
df.drop(columns=['res_price_sqr'], inplace=True)

## 4. Feature Engineering
The next stage involves creating new features based either on existing data or external knowledge. This helps the model better "understand" the relationships between variables and train more effectively. To evaluate the contribution of a new feature, we will train the model with and without it and compare the results.**

**Lets use the correlation matrix to check what feautures correlate the most with price (res_price)**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df.corr()
target_corr = corr_matrix['res_price'].sort_values(ascending=False)

print(target_corr.head(4))

**Let's test this idea with a new feature called total_rooms, which sums the number of bathrooms and bedrooms.**

In [ ]:
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

In [ ]:
X_with = df.drop(columns=['res_price'])
X_without = X_with.drop(columns=['total_rooms'])
y = df['res_price']

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=40)

r2_with = cross_val_score(model, X_with, y, scoring='r2', cv=5).mean()
r2_without = cross_val_score(model, X_without, y, scoring='r2', cv=5).mean()

print(f"XGBoost R² with total_rooms: {r2_with:.4f}")
print(f"XGBoost R² no total_rooms: {r2_without:.4f}")


**It’s rejected because, even slightly, it worsens the model’s prediction performance.**

In [ ]:
df.drop(columns=["total_rooms"], inplace=True)

**We will group the areas into 6 broader regions: North, South, East, West, Center, and Piraeus.**

In [ ]:
regions_map = {
    'voreia': [
        'marousi', 'cholargos', 'filothei-psychiko', 'ekali', 'vrilissia', 'drosia', 'dionysos','nea-ionia'
    ],
    'notia': [
        'glyfada', 'alimos', 'elliniko-argyroupoli','argyroupoli', 'vari-voula-vouliagmeni', 'vari','varkiza', 'lavreotiki','agios-dimitrios','ilioupoli','nea-smyrni','palaio-faliro','moschato-tavros'
    ],
    'anatolika': [
        'spata', 'kropia', 'rafina-pikermi','acharnes'
    ],
    'dytika': [
        'peristeri', 'aigaleo', 'agioi-anargyroi','ilion'
    ],
    'kentro': [
        'athina', 'pagkrati','galatsi', 'dafni', 'kallithea', 'zografou', 'chalandri', 'vyronas'
    ],
    'pireas': [
        'keratsini-drapetsona', 'argyroupoli', 'nikaia', 'peiraias','kaminia'
    ]
}


In [ ]:
for group_name, areas in regions_map.items():
    existing_areas = [area for area in areas if area in df.columns]
    df[group_name] = df[existing_areas].sum(axis=1)

print(df[['voreia', 'notia', 'anatolika', 'dytika', 'kentro', 'pireas']].sum())

In [ ]:
X_with = df.drop(columns=['res_price'])
X_without = X_with.drop(columns=['voreia', 'notia', 'anatolika', 'dytika', 'kentro', 'pireas'])
y = df['res_price']

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

r2_with = cross_val_score(model, X_with, y, scoring='r2', cv=5).mean()
r2_without = cross_val_score(model, X_without, y, scoring='r2', cv=5).mean()

print(f"XGBoost R² regions: {r2_with:.5f}")
print(f"XGBoost R² no regions: {r2_without:.5f}")

**The room_density field combines the total number of rooms and divides it by the property's square meters. It represents the density of the house and, as we'll see later, significantly improves the model's performance.**

In [ ]:
df['room_density'] = (df['bedrooms'] + df['bathrooms']) / df['res_sqr']


In [ ]:
X_with = df.drop(columns=['res_price'])
X_without = X_with.drop(columns=['room_density'])
y = df['res_price']

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

r2_with = cross_val_score(model, X_with, y, scoring='r2', cv=5).mean()
r2_without = cross_val_score(model, X_without, y, scoring='r2', cv=5).mean()

print(f"XGBoost R² room_denstiy: {r2_with:.5f}")
print(f"XGBoost R² no room_density: {r2_without:.5f}")

In [ ]:
df.drop(columns=['room_density'], inplace=True)

**We introduce a new field, luxury_index, which indicates the amenities and features of the house.**

In [ ]:
df['luxury_index'] = df['fireplace'] + df['cooling'] + df['safe_door'] + df['solar'] + df['excellent'] + df['brand_new']

In [ ]:
X_with = df.drop(columns=['res_price'])
X_without = X_with.drop(columns=['luxury_index'])
y = df['res_price']

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

r2_with = cross_val_score(model, X_with, y, scoring='r2', cv=5).mean()
r2_without = cross_val_score(model, X_without, y, scoring='r2', cv=5).mean()

print(f"XGBoost R² luxury_index: {r2_with:.5f}")
print(f"XGBoost R² no luxury_index: {r2_without:.5f}")

In [ ]:
df['is_coastal'] = df['palaio-faliro'] + df['moschato-tavros'] + df['vari-voula-vouliagmeni'] + df['peiraias'] + df['alimos'] + df['glyfada'] + df['elliniko-argyroupoli'] + df['vari'] + df['varkiza'] + df['rafina-pikermi'] + df['lavreotiki']


In [ ]:
X_with = df.drop(columns=['res_price'])
X_without = X_with.drop(columns=['is_coastal'])
y = df['res_price']

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

r2_with = cross_val_score(model, X_with, y, scoring='r2', cv=5).mean()
r2_without = cross_val_score(model, X_without, y, scoring='r2', cv=5).mean()

print(f"XGBoost R² is_coastal: {r2_with:.5f}")
print(f"XGBoost R² no is_coastal: {r2_without:.5f}")

In [ ]:
df.drop(columns=['is_coastal'], inplace=True)

## 5. Outlier removal

The final stage before training the model is removing all outliers. At the end, we will compare the results before and after their removal.

In [ ]:
df_original = df.copy()

In [ ]:
def plot_house_age_with_counts(data, title):
    plt.figure(figsize=(12, 6))
    ax = sns.histplot(data['house_age'], bins=30, kde=False, color='skyblue')
    plt.title(title)
    plt.xlabel('House Age')
    plt.ylabel('Count')
    
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.text(p.get_x() + p.get_width()/2, height + 3, int(height), ha='center', fontsize=9)
    
    plt.show()

plot_house_age_with_counts(df, "House Age Distribution BEFORE Removing Outliers")

In [ ]:
df = df[(df['house_age'] >= 0) & (df['house_age'] <= 70)].copy()
print(f"New total nubmer of houses: {df.shape[0]}")

In [ ]:
def plot_price_distribution_with_stats(data, title):
    plt.figure(figsize=(12, 6))
    ax = sns.histplot(data['res_price'], bins=30, kde=False, color='coral')
    plt.title(title)
    plt.xlabel('Residential Price')
    plt.ylabel('Count')

    ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

    avg_price = data['res_price'].mean()
    p95_price = data['res_price'].quantile(0.95)

    ax.axvline(avg_price, color='blue', linestyle='--', linewidth=2, label=f'Average Price: {avg_price:,.0f}')
    ax.axvline(p95_price, color='green', linestyle='--', linewidth=2, label=f'95th Percentile: {p95_price:,.0f}')

    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.text(p.get_x() + p.get_width()/2, height + 3, int(height), ha='center', fontsize=9)

    plt.legend()
    plt.show()

# Call the function to plot
plot_price_distribution_with_stats(df, "Residential Price Distribution with Average and 95th Percentile")


In [ ]:
p95_price = df['res_price'].quantile(0.95)

num_above_p95 = df[df['res_price'] > p95_price].shape[0]
print(f"Number of properties with price above the 95th percentile ({p95_price:,.0f}): {num_above_p95}")

num_below_or_equal_p95 = df[df['res_price'] <= p95_price].shape[0]
print(f"Number of properties with price below or equal to the 95th percentile ({p95_price:,.0f}): {num_below_or_equal_p95}")

**As we observe from the chart, the number of properties priced above the 95th percentile (€891,200) is 740.
The number of properties priced at or below the 95th percentile (€891,200) is 14,051.
This means that 95% of the properties cost up to €891,200, while the remaining 5% are more expensive properties priced above this threshold. Obviously, we will remove these high-priced properties as outliers.**

In [ ]:
df = df[df['res_price'] <= p95_price].copy()
print(f"Νέος αριθμός ακινήτων: {df.shape[0]}")

**Κρατάμε μόνο τα σπίτια με αξία μεγαλύτερη των 30000 ευρώ**

In [ ]:
df = df[df['res_price'] >= 30000]
print(f"New total nubmer of houses: {df.shape[0]}")

In [ ]:
def plot_res_sqr_distribution_10(data, title):
    plt.figure(figsize=(16, 8))
    
    bins = np.arange(0, 1700 + 10, 10)
    
    ax = sns.histplot(data['res_sqr'], bins=bins, kde=False, color='mediumseagreen')
    plt.title(title, fontsize=16)
    plt.xlabel('Εμβαδόν (τ.μ.)', fontsize=14)
    plt.ylabel('Αριθμός Ακινήτων', fontsize=14)
    
    plt.xlim(0, 1700)
    ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    
    for p in ax.patches:
        if p.get_x() + p.get_width() <= 1700:
            height = p.get_height()
            if height > 0:
                ax.text(p.get_x() + p.get_width()/2, height + 3, int(height), ha='center', fontsize=8, rotation=90)
    
    plt.tight_layout()
    plt.show()

plot_res_sqr_distribution_10(df, "Res_sqr Distribution BEFORE Removing Outliers")



**Properties with less than 20 sqm or more than 400 sqm will be removed as outliers.**

In [ ]:
df = df[(df['res_sqr'] >= 20) & (df['res_sqr'] < 400)]
print(f"New total nubmer of houses: {df.shape[0]}")

In [ ]:
columns = ['res_price', 'res_sqr', 'house_age']

print("Comparison of Statistics Before and After Outlier Removal:\n")

for col in columns:
    print(f"Field: {col}")
    print(f"  Before -> Size: {df_original.shape[0]:,}, Min: {df_original[col].min():,.2f}, Max: {df_original[col].max():,.2f}, Mean: {df_original[col].mean():,.2f}")
    print(f"  After  -> Size: {df.shape[0]:,}, Min: {df[col].min():,.2f}, Max: {df[col].max():,.2f}, Mean: {df[col].mean():,.2f}")
    print()


## 6. Model Training
    At first we use different Regression models like Ridge, Lasso, SGDRegressor etc.
    The best result we get is R^2 = 0.74 with RMSE = 89450 From Ridge. We will try RandomForest and XGBoost in order to get improved perofrmance

In [ ]:
X = df.drop(columns=['res_price'])
y = df['res_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define models
models = {
    "Random Forest": RandomForestRegressor(
        n_estimators=300,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ),
    "XGBoost": XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1
    )
}

# Evaluate models
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results.append({
        'Model': name,
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2),
        'R²': round(r2, 4)
    })

results_df = pd.DataFrame(results).sort_values(by='R²', ascending=False)
print("Performance Comparison:")
print(results_df)


**As seen above XGBoost provides an improved result with R^2 = 0.79 and RMSE = 80106. Lets play with its parameters to get an even better result.**

In [ ]:
best_params = {
    'n_estimators': 800,
    'learning_rate': 0.02,
    'max_depth': 8,
    'subsample': 0.85,
    'colsample_bytree': 0.7,
    'gamma': 0,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'random_state': 42,
    'n_jobs': -1
}

# Four variations around the best parameters
param_variations = [
    {'learning_rate': 0.015, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.75},
    {'learning_rate': 0.025, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.65},
    {'learning_rate': 0.02, 'max_depth': 8, 'subsample': 0.85, 'colsample_bytree': 0.7}, 
    {'learning_rate': 0.03, 'max_depth': 8, 'subsample': 0.87, 'colsample_bytree': 0.72},
]

best_rmse = float('inf')
best_r2 = float('inf')
best_config = None

for idx, variation in enumerate(param_variations, 1):
    params = best_params.copy()
    params.update(variation)

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    if rmse < best_rmse:
        best_rmse = rmse
        best_r2 = r2
        best_config = variation

print("Best variation found:")
print(best_config)
print(f"RMSE: {best_rmse:.2f}")
print(f"R^2: {best_r2:.2f}")

## 7. Conclusion
By carefully preprocessing and analyzing the available data, you can develop a predictive model that provides valuable insights. The key is to find the best fit between the model and the data to generate accurate predictions. Predicting house prices is inherently challenging, especially in today’s rapidly changing market. Nevertheless, building a reliable baseline model for home valuation is crucial, as it can potentially assist buyers, sellers, and real estate professionals in making more informed decisions.

Thank you for investing your time in this notebook. Your feedback is always welcome!

**Ioannis Giakisikloglou**